# calculating return values and periods using climextremes

In [1]:
import requests 
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime

import climextremes as climext

import matplotlib.pyplot as plt
%matplotlib inline

## extracting time series from netcdf file

In [2]:
ds = xr.open_dataset("./data/pr_Amon_NorESM1-M_historical_r1i1p1_185001-200512.nc")
print(ds)

<xarray.Dataset>
Dimensions:    (time: 1872, bnds: 2, lat: 96, lon: 144)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2005-12-16 12:00:00
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    pr         (time, lat, lon) float32 ...
Attributes: (12/26)
    institution:            Norwegian Climate Centre
    institute_id:           NCC
    experiment_id:          historical
    source:                 NorESM1-M 2011  atmosphere: CAM-Oslo (CAM4-Oslo-n...
    model_id:               NorESM1-M
    forcing:                GHG, SA, Oz, Sl, Vl, BC, OC
    ...                     ...
    table_id:               Table Amon (27 April 2011) a5a1c518f52ae340313ba0...
    title:                  NorESM1-M mo

In [3]:
# converting precipitation to m/day units

ds["pr"].data = ds["pr"].data * 86.4
ds["pr"].attrs["units"] = 'm/day' 

In [4]:
# getting dataset specifically for defined location
# coordinates for berkeley, california

lat = 37.87
lon = 122.27
ds_berk = ds.sel(lon=lon, lat=lat, method='nearest')

In [5]:
# converting to timeseries dataframe

sr = ds_berk['pr'].to_pandas()
sr.index = sr.index.to_datetimeindex()
df = pd.DataFrame({'time':sr.index, 'pr':sr.values})

/var/folders/bz/l7frggbx71n9k19wxh7hvwsm0000gn/T/ipykernel_54699/1416090827.py:4: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  sr.index = sr.index.to_datetimeindex()


In [8]:
ams.to_csv('ams_pr.csv')

## using climextremes to calculate extreme value

In [7]:
# get annual max series (ams)

ams = df.groupby([df.time.dt.year])['pr'].max()
ams = ams.reset_index()
ams

,time,pr
0,1850,0.006327
1,1851,0.005336
2,1852,0.005973
3,1853,0.011385
4,1854,0.006396
...,...,...
151,2001,0.006570
152,2002,0.007323
153,2003,0.005163
154,2004,0.005696


In [37]:
result = climext.fit_gev(np.array(ams.pr), returnPeriod = 10, returnValue = 0.011228, 
                         getFit = True, getParams = True, bootSE = True)

R[write to console]: Warning in log(z) :
R[write to console]:  NaNs produced

R[write to console]: Warning in log(z) :
R[write to console]:  NaNs produced

R[write to console]: Warning in (function (y, x = NULL, locationFun = NULL, scaleFun = NULL,  :
R[write to console]: 
 
R[write to console]:  fit_gev: optimization failed; see 'info' in returned object. You may want to run 'fit_gev()' with argument 'getFit = TRUE' to see more details regarding the attempted optimization. You may also want to try a different optimization method or increase the maximum number of iterations.



In [38]:
result

{'info': {'convergence': array([0], dtype=int32),
  'counts_names': array(['function', 'gradient'], dtype='<U8'),
  'counts': array([82., nan]),
  'message': array(['fitting produced missing or invalid estimates or standard errors'],
        dtype='<U64'),
  'failure': array([1], dtype=int32)},
 'fit': {'call': Rlang( fevd(x = .y ~ 1, data = x, location.fun = locationFun, scale.fun = scaleFun,  ),
  'data.name': array(['.y ~ 1', 'x'], dtype='<U6'),
  'weights': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1